In [18]:
import pandas as pd
import numpy as np
import keras as k
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D

## My Goal/Notes/Ideas

To build a deep learning image classification model can identify pnemonia based on a set of X-ray images.

 - Type I error(FP): predict pnemonia but does not have it.
 - Type II error(FN): has pnemonia but predict no pnemonia.
 
Based on this I feel I should make lungs with pnemonia my target variables so these will be my Y-values.

I should also consider concentrating on accuracy or the F1 score.  Maybe try tweaking a model for each?

## Import data EDA

In [3]:
# Make Training Generator

train_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./256)
generate_train = train_gen.flow_from_directory(directory=r'../data/train', seed=42, 
                                    target_size=(64, 64), 
                                    batch_size=50, 
                                    class_mode='binary',
                                    interpolation='nearest')

# Make Val Generator

val_gen = ImageDataGenerator(rescale=1./256)
generate_val = val_gen.flow_from_directory(directory=r'../data/val', seed=42,
                                   target_size=(256,256),
                                   batch_size=8, 
                                   class_mode='binary',
                                   interpolation='nearest')

# Maket Test Generator

test_gen = ImageDataGenerator(rescale=1./256)
generate_test = test_gen.flow_from_directory(directory=r'../data/test', seed=42,
                                   target_size=(256,256),
                                   batch_size=50, 
                                   class_mode='binary',
                                   interpolation='nearest')

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [20]:
train_image, train_label = next(generate_train)
test_image, test_label = next(generate_test)
val_image, val_label = next(generate_val)

In [35]:
train_image.shape

(50, 64, 64, 3)

In [32]:
train_label.shape

(50,)

In [30]:
print(train_gen.dtype)
print(val_gen.dtype)
print(test_gen.dtype)

float32
float32
float32


## Create FSM

In [57]:
# USING A CNN MODEL!!!

model = tf.keras.models.Sequential()

# Da'model
model.add(Conv2D(10, 3, activation='relu', input_shape=(64,64,3), padding='same'))
model.add(MaxPooling2D(2,2))
model.add(k.layers.Flatten())
model.add(Dense(units=1, activation='sigmoid'))

# Compiler layer
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

# Fitting Da'model
model.fit(train_image, train_label, epochs=10, steps_per_epoch=10) 

Epoch 1/10
10/10 [==============================] - 0s 9ms/step - loss: 0.7858 - accuracy: 0.5800
Epoch 2/10
10/10 [==============================] - 0s 10ms/step - loss: 0.7075 - accuracy: 0.5800
Epoch 3/10
10/10 [==============================] - 0s 10ms/step - loss: 0.5707 - accuracy: 0.7000
Epoch 4/10
10/10 [==============================] - 0s 10ms/step - loss: 0.5327 - accuracy: 0.8200
Epoch 5/10
10/10 [==============================] - 0s 11ms/step - loss: 0.4774 - accuracy: 0.7200
Epoch 6/10
10/10 [==============================] - 0s 10ms/step - loss: 0.3822 - accuracy: 0.9800
Epoch 7/10
10/10 [==============================] - 0s 9ms/step - loss: 0.3806 - accuracy: 0.8800
Epoch 8/10
10/10 [==============================] - 0s 9ms/step - loss: 0.3216 - accuracy: 0.9000
Epoch 9/10
10/10 [==============================] - 0s 9ms/step - loss: 0.2816 - accuracy: 0.9000
Epoch 10/10
10/10 [==============================] - 0s 11ms/step - loss: 0.2763 - accuracy: 0.9200


### Results
The model functions! The accuracy score seems to be REALLY HIGH for a first simple model which makes me think this
model is probably over fit or it needs some kind of adjustment.  The accuracy also changes consistently every time I rerun the model.

In [49]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 64, 64, 10)        280       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 32, 32, 10)        0         
_________________________________________________________________
flatten (Flatten)            (None, 10240)             0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 10241     
Total params: 10,521
Trainable params: 10,521
Non-trainable params: 0
_________________________________________________________________


### Second Model: adding layers

In [58]:
model = tf.keras.models.Sequential()

# Da'model
model.add(Conv2D(10, 3, activation='relu', input_shape=(64,64,3), padding='same'))
model.add(MaxPooling2D(2,2))

# Added newbie layer for this iteration; hope accuracy improves!
model.add(Conv2D(15, 3, activation='relu', input_shape=(63,64,3), padding='same'))
model.add(MaxPooling2D(2,2))

model.add(k.layers.Flatten())
model.add(Dense(units=1, activation='sigmoid'))

# Compiler layer
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

# Fitting Da'model
model.fit(train_image, train_label, epochs=10, steps_per_epoch=10) 

Epoch 1/10
10/10 [==============================] - 0s 17ms/step - loss: 0.6953 - accuracy: 0.5800
Epoch 2/10
10/10 [==============================] - 0s 18ms/step - loss: 0.6759 - accuracy: 0.5800
Epoch 3/10
10/10 [==============================] - 0s 18ms/step - loss: 0.6513 - accuracy: 0.6600
Epoch 4/10
10/10 [==============================] - 0s 15ms/step - loss: 0.6156 - accuracy: 0.9000
Epoch 5/10
10/10 [==============================] - 0s 18ms/step - loss: 0.5787 - accuracy: 0.7200
Epoch 6/10
10/10 [==============================] - 0s 18ms/step - loss: 0.5307 - accuracy: 0.9400
Epoch 7/10
10/10 [==============================] - 0s 20ms/step - loss: 0.4763 - accuracy: 0.9000
Epoch 8/10
10/10 [==============================] - 0s 16ms/step - loss: 0.4242 - accuracy: 0.9000
Epoch 9/10
10/10 [==============================] - 0s 18ms/step - loss: 0.4581 - accuracy: 0.7200
Epoch 10/10
10/10 [==============================] - 0s 18ms/step - loss: 0.3244 - accuracy: 0.9200
